In [197]:
using Revise, DynamicHMC, PosteriorDB, Random, StanLogDensityProblems, JSON, BridgeStan, DataFrames, LinearAlgebra, StatsBase, Distributions
using Plots, ColorSchemes, LaTeXStrings, Statistics, CSV, AdvancedHMC, DynamicObjects
Plots.theme(:default)
const BS = BridgeStan;

In [198]:
pdb = PosteriorDB.database()
model = PosteriorDB.model(pdb, "eight_schools_noncentered")
post = PosteriorDB.posterior(pdb, "eight_schools-eight_schools_noncentered")
impl = PosteriorDB.implementation(model, "stan")
path = PosteriorDB.path(impl)
data = PosteriorDB.dataset(post)
s = PosteriorDB.load(data, String);
bs_model = BS.StanModel(stan_file=path, data=s);
ref = PosteriorDB.reference_posterior(post)
df=DataFrame(PosteriorDB.load(ref));

┌ Warning: Loading a shared object '/home/meenaljhajharia/.julia/artifacts/95c3074bd791a2f28857088639b48ca1b786459a/posteriordb-0.4.0/posterior_database/models/stan/eight_schools_noncentered_model.so' which is already loaded.
│ If the file has changed since the last time it was loaded, this load may not update the library!
└ @ BridgeStan /home/meenaljhajharia/.julia/packages/BridgeStan/nCPwJ/src/model.jl:51


In [214]:
using TransformVariables
using TransformedLogDensities, LogDensityProblemsAD
t = as((μ = asℝ, σ = asℝ₊, τ = asℝ₊, θs = as(Array, 8)))
dimension(t)
lp = StanProblem(bs_model);
P = TransformedLogDensity(t, lp)
∇P = ADgradient(:ForwardDiff, lp);

D = 10;
initial_θ = rand(Uniform(-2, 2), D);
n_samples, n_adapts = 8000, 2000;
metric = DiagEuclideanMetric(D);
hamiltonian = Hamiltonian(metric, lp);
initial_ϵ = find_good_stepsize(hamiltonian, initial_θ);
integrator = Leapfrog(initial_ϵ);
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator);
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.95, integrator));
samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; progress=true);

Sampling  13%|████                           |  ETA: 0:00:01
  iterations:                                   1032
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.01
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              0.9627831560454742
  log_density:                                  -6.482228142840333
  hamiltonian_energy:                           8.86534823842792
  hamiltonian_energy_error:                     0.04397939798452555
  max_hamiltonian_energy_error:                 -0.16452010252076832
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.46213311171572213
  nom_step_size:                                0.46213311171572213
  is_adapt:                                     true



















Sampling  34%|██████████▋                    |  ETA: 0:00:00
  iterations:                                   2740
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              0.9689961542166717
  log_density:                                  -3.9350137834644814
  hamiltonian_energy:                           9.23985541075146
  hamiltonian_energy_error:                     -0.011110994448685574
  max_hamiltonian_energy_error:                 0.09742298527752169
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.2891197669271062
  nom_step_size:                                0.2891197669271062
  is_adapt:                                     false



















Sampling  58%|█████████████████▉             |  ETA: 0:00:00
  iterations:                                   4602
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      15
  is_accept:                                    true
  acceptance_rate:                              0.9697213388404239
  log_density:                                  -10.124670009132041
  hamiltonian_energy:                           13.144305202379245
  hamiltonian_energy_error:                     0.06029475365978065
  max_hamiltonian_energy_error:                 0.06675013497055282
  tree_depth:                                   4
  numerical_error:                              false
  step_size:                                    0.2891197669271062
  nom_step_size:                                0.2891197669271062
  is_adapt:                                     false



















Sampling  81%|█████████████████████████      |  ETA: 0:00:00
  iterations:                                   6463
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      15
  is_accept:                                    true
  acceptance_rate:                              0.997750327902989
  log_density:                                  -6.180004059969118
  hamiltonian_energy:                           8.443458818275149
  hamiltonian_energy_error:                     0.009509702567038758
  max_hamiltonian_energy_error:                 -0.05227724645316201
  tree_depth:                                   4
  numerical_error:                              false
  step_size:                                    0.2891197669271062
  nom_step_size:                                0.2891197669271062
  is_adapt:                                     false



















Sampling 100%|███████████████████████████████| Time: 0:00:00
  iterations:                                   8000
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      15
  is_accept:                                    true
  acceptance_rate:                              0.8792152637710148
  log_density:                                  -5.487238862041923
  hamiltonian_energy:                           6.818726232451577
  hamiltonian_energy_error:                     0.13740340665458373
  max_hamiltonian_energy_error:                 0.36021956480921435
  tree_depth:                                   4
  numerical_error:                              false
  step_size:                                    0.2891197669271062
  nom_step_size:                                0.2891197669271062
  is_adapt:                                     false
  mass_matrix:                   

┌ Info: Finished 8000 sampling steps for 1 chains in 0.546865159 (s)
│   h = Hamiltonian(metric=DiagEuclideanMetric([0.9573192979290929, 0.8479 ...]), kinetic=GaussianKinetic())
│   κ = HMCKernel{AdvancedHMC.FullMomentumRefreshment, Trajectory{MultinomialTS, Leapfrog{Float64}, GeneralisedNoUTurn{Float64}}}(AdvancedHMC.FullMomentumRefreshment(), Trajectory{MultinomialTS}(integrator=Leapfrog(ϵ=0.289), tc=GeneralisedNoUTurn{Float64}(10, 1000.0)))
│   EBFMI_est = 0.9605831934526045
│   average_acceptance_rate = 0.9626434753186711
└ @ AdvancedHMC /home/meenaljhajharia/.julia/packages/AdvancedHMC/P0nla/src/sampler.jl:246


In [215]:
constrained_draws = hcat([vcat(col...) for col in eachcol(df)]...)
unc_sample_array = vcat([
    param_unconstrain(bs_model, collect(row))' for row in eachrow(constrained_draws)
]...);

In [216]:
lol = hcat(samples...)'
conhmc = vcat([
    param_constrain(bs_model, collect(row))' for row in eachrow(lol)
]...);

In [217]:
mean(unc_sample_array, dims=1)

1×10 Matrix{Float64}:
 6.1505  4.93958  3.90591  4.79602  …  6.31717  4.884  4.41052  0.808081

In [218]:
mean(conhmc, dims=1)

1×10 Matrix{Float64}:
 0.302484  0.0937178  -0.0986124  …  0.363376  0.080201  4.4096  3.68547

In [219]:
samples

8000-element Vector{Vector{Float64}}:
 [0.963432678308572, -0.12668072503992056, 2.4528186669813787, 1.677265566166034, -0.2426708269469322, -1.474201209786235, 0.7868956548009889, 0.07685678971551635, 2.581029577238704, 1.6860354695458857]
 [0.963432678308572, -0.12668072503992056, 2.4528186669813787, 1.677265566166034, -0.2426708269469322, -1.474201209786235, 0.7868956548009889, 0.07685678971551635, 2.581029577238704, 1.6860354695458857]
 [-0.21818503925287314, 1.3545169499708658, -2.2838570196936696, -1.605005212877185, 0.7209229456564784, 1.098808031514759, -0.16742983801689965, -0.4485779598957871, -1.367448867808196, 1.8189273592447772]
 [0.7402931082423165, -1.5150186444072067, 2.280473371718401, 1.405308977660168, -1.1157509452128322, -1.1268654611122335, 0.6024235526227872, 0.5293984076366991, 0.12039436452221583, -0.2717522217723567]
 [1.3954592767590774, -2.0480776061934476, 1.426408744459492, 1.07019393241283, 0.07623550675109592, -1.0626678964921359, 0.6183037127860472, 0.